### CodeCarbon

In [ ]:
import tensorflow as tf
from codecarbon import EmissionsTracker

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
    ]
)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

with EmissionsTracker(
    project_name="model-distillation", 
    experiment_id="123", 
    tracking_mode="process",
    output_dir="metrics/emissions"
    ) as tracker:
    model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
    model.fit(x_train, y_train, epochs=10)

/Users/hendriksippel/Documents/Repositories/cbs-thesis-efficient-llm-distillation/.venv/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[codecarbon INFO @ 09:37:51] [setup] RAM Tracking...
[codecarbon INFO @ 09:37:51] [setup] CPU Tracking...
[codecarbon WARNING @ 09:37:51] No CPU tracking mode found. Falling back on CPU constant mode. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 09:37:51] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 09:37:51] [setup] GPU Tracking...
[codecarbon INFO @ 09:37:51] No GPU found.
[codecarbon INFO @ 09:37:51] >>> Tracker's metadata:
[codecarbon INFO @ 09:37:51]   Platform system: macOS-15.3-arm64-arm-64bit
[codecarbon INFO @ 09:37:51]   Python ve

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8638 - loss: 0.4724
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9543 - loss: 0.1494
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9669 - loss: 0.1116
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9732 - loss: 0.0859
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9768 - loss: 0.0743
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9794 - loss: 0.0651
Epoch 7/10
 355/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9829 - loss: 0.0542

[codecarbon INFO @ 09:38:06] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.11040115356445312 W
[codecarbon INFO @ 09:38:06] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 09:38:06] 0.000021 kWh of electricity used since the beginning.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9814 - loss: 0.0574
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9838 - loss: 0.0491
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9851 - loss: 0.0459
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9864 - loss: 0.0419


[codecarbon INFO @ 09:38:15] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.03935050964355469 W
[codecarbon INFO @ 09:38:15] Energy consumed for all CPUs : 0.000033 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 09:38:15] 0.000033 kWh of electricity used since the beginning.
/Users/hendriksippel/Documents/Repositories/cbs-thesis-efficient-llm-distillation/.venv/lib/python3.11/site-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


In [18]:
print(f"Tracked final energy consumption: {tracker._total_energy.kWh} kWh")
print(f"Tracked final CO2 emissions: {tracker.final_emissions} kgCO2")

Tracked final energy consumption: 3.342663552604511e-05 kWh
Tracked final CO2 emissions: 5.0691492775247405e-06 kgCO2


### Ollama

In [1]:
import requests

MODEL = "llama3.2:1b"

In [2]:
# The following code also calls the appropriate Ollama endpoint and activates the model.

url = "http://localhost:11434/api/chat"

def ollama(model, prompt):
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["message"]["content"]
    else:
        return "Error"

response = ollama(
    model=MODEL,
    prompt="How often does the letter a appear in banana?"
    )

print(response)

The letter "a" appears twice in the word "banana".


In [4]:
# To list all actively running models, you can use the following code:
!ollama ps

NAME           ID              SIZE      PROCESSOR    UNTIL              
llama3.2:1b    baf6a787fdff    2.8 GB    100% GPU     4 minutes from now    


In [ ]:
# To stop the Ollama server, execute the following command:
!ollama stop llama3.2:1b

# Confirmation
!ollama ps

NAME    ID    SIZE    PROCESSOR    UNTIL 
